In [10]:
import numpy as np
# from multiprocessing import Pool  # , Process
from cerebros.simplecerebrosrandomsearch.simple_cerebros_random_search\
    import SimpleCerebrosRandomSearch
import pendulum
import pandas as pd
import tensorflow as tf
from cerebros.units.units import DenseUnit
from cerebros.denseautomlstructuralcomponent.dense_automl_structural_component\
    import zero_7_exp_decay, zero_95_exp_decay, simple_sigmoid
from ast import literal_eval

In [11]:
activation = 'relu'
predecessor_level_connection_affinity_factor_first = 2.0
predecessor_level_connection_affinity_factor_main = 0.97
max_consecutive_lateral_connections = 5
p_lateral_connection = 0.97
num_lateral_connection_tries_per_unit = 2
learning_rate = 0.001
epochs = 10
batch_size = 100

In [13]:

raw_data = pd.read_csv('ames.csv')
needed_cols = [
    col for col in raw_data.columns if raw_data[col].dtype != 'object']
data_numeric = raw_data[needed_cols].fillna(0).astype(float)
label = raw_data.pop('price')

data_np = data_numeric.values

# np.save('label', label, allow_pickle=True)
# np.save('data', data_np, allow_pickle=True)

TIME = pendulum.now(tz='America/New_York').__str__()[:16]\
    .replace('T', '_')\
    .replace(':', '_')\
    .replace('-', '_')
PROJECT_NAME = f'{TIME}_cerebros_auto_ml_test'

# data_np = np.load('data.npy', allow_pickle=True)
# labels = np.load('labels.npy', allow_pickle=True)

tensor_x =\
    tf.constant(data_np)

training_x = [tensor_x]

INPUT_SHAPES = [training_x[i].shape[1] for i in np.arange(len(training_x))]


#alcohol_labels = tf.constant(pd.get_dummies(pd.cut(white['alcohol'], np.arange(
#    white['alcohol'].min(), white['alcohol'].max()))).values)
# quality_labels = tf.constant(pd.get_dummies(white['quality']).values)

train_labels = [label.values]

# OUTPUT_SHAPES = [1]  # [train_labels[i].shape[1]
# for i in np.arange(len(train_labels))]


In [ ]:

# TIME = pendulum.now(tz='America/New_York').__str__()[:16]\
#     .replace('T', '_')\
#     .replace(':', '_')\
#     .replace('-', '_')
# PROJECT_NAME = f'{TIME}_cerebros_auto_ml_test'

# data_np = np.load('data.npy', allow_pickle=True)
# labels = np.load('labels.npy', allow_pickle=True)

# tensor_x =\
#     tf.constant(data_np)

# training_x = [tensor_x]

INPUT_SHAPES = [training_x[i].shape[1] for i in np.arange(len(training_x))]


# #alcohol_labels = tf.constant(pd.get_dummies(pd.cut(white['alcohol'], np.arange(
# #    white['alcohol'].min(), white['alcohol'].max()))).values)
# # quality_labels = tf.constant(pd.get_dummies(white['quality']).values)

# train_labels = [label.values]

OUTPUT_SHAPES = [1]  # [train_labels[i].shape[1]
# # for i in np.arange(len(train_labels))]

meta_trial_number = str(int(np.random.random() * 10 ** 12))


cerebros_automl = SimpleCerebrosRandomSearch(
    unit_type=DenseUnit,
    input_shapes=INPUT_SHAPES,
    output_shapes=OUTPUT_SHAPES,
    training_data=training_x,
    labels=train_labels,
    validation_split=0.35,
    direction='minimize',
    metric_to_rank_by='val_root_mean_squared_error',
    minimum_levels=2,
    maximum_levels=7,
    minimum_units_per_level=1,
    maximum_units_per_level=4,
    minimum_neurons_per_unit=1,
    maximum_neurons_per_unit=4,
    activation=activation,
    final_activation=None,
    number_of_architecture_moities_to_try=7,
    number_of_tries_per_architecture_moity=1,
    minimum_skip_connection_depth=1,
    maximum_skip_connection_depth=7,
    predecessor_level_connection_affinity_factor_first=predecessor_level_connection_affinity_factor_first,
    predecessor_level_connection_affinity_factor_first_rounding_rule='ceil',
    predecessor_level_connection_affinity_factor_main=predecessor_level_connection_affinity_factor_main,
    predecessor_level_connection_affinity_factor_main_rounding_rule='ceil',
    predecessor_level_connection_affinity_factor_decay_main=zero_7_exp_decay,
    seed=8675309,
    max_consecutive_lateral_connections=max_consecutive_lateral_connections,
    gate_after_n_lateral_connections=3,
    gate_activation_function=simple_sigmoid,
    p_lateral_connection=p_lateral_connection,
    p_lateral_connection_decay=zero_95_exp_decay,
    num_lateral_connection_tries_per_unit=num_lateral_connection_tries_per_unit,
    learning_rate=learning_rate,
    loss='mse',
    metrics=[tf.keras.metrics.RootMeanSquaredError()],
    epochs=epochs,
    project_name=f"{PROJECT_NAME}_meta_{meta_trial_number}",
    # use_multiprocessing_for_multiple_neural_networks=False,  # pull this param
    model_graphs='model_graphs',
    batch_size=batch_size,
    meta_trial_number=meta_trial_number)
val_root_mean_squared_error = cerebros_automl.run_random_search()


In [ ]:
print(val_root_mean_squared_error)